In [1]:
# Get L2 Revenue and post it to a database (csv in github for now)
import pandas as pd
import sys
import time
import json
sys.path.append("../helper_functions")
import web3py_utils as w3py
import duneapi_utils as du
from web3 import Web3
from datetime import datetime
sys.path.pop()

import os

In [2]:
# https://github.com/ethereum-optimism/optimism/blob/b86522036ad11a91de1d1dadb6805167add83326/specs/predeploys.md?plain=1#L50

# Name, contract
fee_vaults = [
    ['SequencerFeeVault','0x4200000000000000000000000000000000000011'],
    ['BaseFeeVault','0x4200000000000000000000000000000000000019'],
    ['L1FeeVault','0x420000000000000000000000000000000000001A'],
]

# Aiming to eventually read from superchain-resitry + some non-superchain static adds
chains_rpcs = pd.read_csv('inputs/chain_metadata.csv')
# print(chains_rpcs.columns)
chains_rpcs = chains_rpcs[~(chains_rpcs['rpc_url'] == ' ') & ~(chains_rpcs['op_based_version'].str.contains('legacy'))]
# chains_rpcs = chains_rpcs.values.tolist()
print(chains_rpcs)

             chain_name op_based_version has_mods is_law_of_chains  \
0            op mainnet          bedrock       no              yes   
1                  base          bedrock       no              yes   
2          zora network          bedrock       no               no   
3  public goods network          bedrock       no               no   
5                 opbnb          bedrock      yes               no   
6         manta pacific          bedrock      yes               no   
7            loot chain          bedrock      yes               no   
8         kroma network          bedrock      yes               no   
9                vienna          bedrock      yes               no   

  raas_deployer                                  rpc_url  \
0                            https://mainnet.optimism.io   
1                              https://mainnet.base.org/   
2       conduit                 https://rpc.zora.energy/   
3       conduit         https://rpc.publicgoods.network/   

In [3]:
# Calculate the method signature hash
method_signature = "totalProcessed()"
method_id = Web3.keccak(text=method_signature)[:4].hex()
# Verify the method ID
print(f"Method ID: {method_id}")

Method ID: 0x84411d65


In [4]:
data_arr = []

for index, chain in chains_rpcs.iterrows():
    chain_name = chain['chain_name']
    print(chain_name)
    rpc = chain['rpc_url']
    # gas_token = chain['gas_token']
    # da_layer = chain['da_layer']
    # is_superchain_registry = chain['superchain_registry']

    try:
        w3_conn = Web3(Web3.HTTPProvider(rpc))
        # Get the timestamp of the latest block
        block_timestamp = w3_conn.eth.get_block('latest').timestamp
        block_number = w3_conn.eth.get_block('latest').number
        # Convert the UNIX timestamp to a human-readable format
        block_time = datetime.utcfromtimestamp(block_timestamp).strftime('%Y-%m-%d %H:%M:%S')

        for vault in fee_vaults:
            vault_name = vault[0]
            vault_address = vault[1]
            # Call the function directly using eth_call
            response = w3_conn.eth.call({
                'to': vault_address,
                'data': method_id
            })
            wei_balance = w3_conn.eth.get_balance(vault_address)
            # Decode the result (assuming the function returns a uint256)
            proxy_processed_wei = Web3.to_int(hexstr=response.hex())
            
            alltime_revenue_wei = proxy_processed_wei+wei_balance
            alltime_revenue_native = alltime_revenue_wei/1e18

            print(chain_name + ' | ' + vault_name + ': ' \
                + str(proxy_processed_wei) + ' | bal: ' + str(wei_balance)\
                + ' | total eth: ' + str( (alltime_revenue_native) )
                )
            
            tmp = pd.DataFrame(
                    [[block_time, block_number, chain_name, vault_name, vault_address, alltime_revenue_native]]#, gas_token, da_layer, is_superchain_registry]]
                    ,columns =['block_time','block_number','chain_name','vault_name','vault_address','alltime_revenue_native']#,'gas_token','da_layer','is_superchain_registry']
                    )
            data_arr.append(tmp)
            time.sleep(1)
    except:
        continue

data_df = pd.concat(data_arr)

op mainnet
op mainnet | SequencerFeeVault: 1489676782078703420819 | bal: 21845098692594353432 | total eth: 1511.521880771298
op mainnet | BaseFeeVault: 613440458297089430527 | bal: 10507441120774022626 | total eth: 623.9478994178634
op mainnet | L1FeeVault: 4210547361710568971100 | bal: 21664050314605048886 | total eth: 4232.2114120251745
base
base | SequencerFeeVault: 1905192317840911911061 | bal: 309118822228053774227 | total eth: 2214.311140068966
base | BaseFeeVault: 442281139106418194815 | bal: 47036342537942891415 | total eth: 489.31748164436107
base | L1FeeVault: 1567676667751681489641 | bal: 418418903701453887861 | total eth: 1986.0955714531353
zora network
zora network | SequencerFeeVault: 256842974805031609748 | bal: 2506375345649946340 | total eth: 259.3493501506816
zora network | BaseFeeVault: 0 | bal: 43977897166264 | total eth: 4.3977897166264e-05
zora network | L1FeeVault: 208754535130387528966 | bal: 808748117930467790 | total eth: 209.563283248318
public goods network


In [5]:

file_path = 'outputs/all_time_revenue_data.csv'

# Check if the file exists
if os.path.exists(file_path):
    # If file exists, append without writing the header
    data_df.to_csv(file_path, mode='a', header=False, index=False)
else:
    # If file doesn't exist, create it and write the header
    data_df.to_csv(file_path, mode='w', header=True, index=False)

In [6]:
# Write to Dune Table

dune_df = pd.read_csv(file_path)

print(dune_df.sample(5))

du.write_dune_api_from_pandas(dune_df, 'op_stack_chains_cumulative_revenue_snapshots',\
                             'Snapshots of All-Time (cumulative) revenue for fee vaults on OP Stack Chains. Pulled from RPCs - metadata in op_stack_chains_chain_rpc_metdata')
du.write_dune_api_from_pandas(chains_rpcs, 'op_stack_chains_chain_rpc_metdata',\
                             'Chain metadata - used to join with op_stack_chains_cumulative_revenue_snapshots')

                block_time  block_number            chain_name  \
3132   2023-09-15 14:00:31       3998542                  base   
14477  2023-10-13 18:00:27       4056650  public goods network   
14337  2023-10-13 09:59:59        296724         manta pacific   
13592  2023-10-11 15:00:07       5171284          zora network   
9455   2023-10-02 00:00:29       3549051  public goods network   

              vault_name                               vault_address  \
3132   SequencerFeeVault  0x4200000000000000000000000000000000000011   
14477         L1FeeVault  0x420000000000000000000000000000000000001A   
14337  SequencerFeeVault  0x4200000000000000000000000000000000000011   
13592         L1FeeVault  0x420000000000000000000000000000000000001A   
9455          L1FeeVault  0x420000000000000000000000000000000000001A   

       alltime_revenue_native gas_token   da_layer is_superchain_registry  
3132              1805.672797       ETH   ethereum                    yes  
14477             